TITLE

INTRODUCTION

PRELIMINARY DATA ANALYSIS

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [4]:
audit_data <- read_csv("/home/jupyter/Group_35_project_proposal/audit_risk.csv")
audit_data

Warning message:
“Duplicated column names deduplicated: 'Score_B' => 'Score_B_1' [11]”
Parsed with column specification:
cols(
  .default = col_double(),
  LOCATION_ID = col_character()
)

See spec(...) for full column specifications.



Sector_score,LOCATION_ID,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,⋯,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3.89,23,4.18,0.6,2.508,2.50,0.2,0.500,6.68,5.0,⋯,0.4,0,0.2,0.0,2.4,8.5740,0.4,0.5,1.71480,1
3.89,6,0.00,0.2,0.000,4.83,0.2,0.966,4.83,5.0,⋯,0.4,0,0.2,0.0,2.0,2.5540,0.4,0.5,0.51080,0
3.89,6,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,⋯,0.4,0,0.2,0.0,2.0,1.5480,0.4,0.5,0.30960,0
3.89,6,0.00,0.2,0.000,10.80,0.6,6.480,10.80,6.0,⋯,0.4,0,0.2,0.0,4.4,17.5300,0.4,0.5,3.50600,1
3.89,6,0.00,0.2,0.000,0.08,0.2,0.016,0.08,5.0,⋯,0.4,0,0.2,0.0,2.0,1.4160,0.4,0.5,0.28320,0
3.89,6,0.00,0.2,0.000,0.83,0.2,0.166,0.83,5.0,⋯,0.4,0,0.2,0.0,2.0,2.1560,0.4,0.5,0.43120,0
3.89,7,1.10,0.4,0.440,7.41,0.4,2.964,8.51,5.0,⋯,0.4,0,0.2,0.0,3.2,31.7740,0.4,0.5,6.35480,1
3.89,8,8.50,0.6,5.100,12.03,0.6,7.218,20.53,5.5,⋯,0.4,0,0.2,0.0,4.2,18.0340,0.4,0.5,3.60680,1
3.89,8,8.40,0.6,5.040,11.05,0.6,6.630,19.45,5.5,⋯,0.4,0,0.2,0.0,4.2,17.2060,0.4,0.5,3.44120,1


METHODS

EXPECTED OUTCOMES / SIGNIFICANCE